In [5]:
import os
import numpy as np
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from tkinter import Tk, filedialog

# コンスタントの定義
VU_REF_DB = -18.0
VU_MAX = 0.0
REF_LEVEL = 10**(VU_REF_DB / 20)
TARGET_LEVEL = 1.0  # 0 VUの基準を1に設定

# ファイル選択ダイアログを表示してファイルを選択する
root = Tk()
root.withdraw()  # メインウィンドウを表示しない
file_path = filedialog.askopenfilename(filetypes=[("WAV files", "*.wav")])
root.destroy()  # Tkinterウィンドウを破棄する

# オーディオファイルの読み込み
if file_path:
    rate, data = wav.read(file_path)
    if data.ndim == 1:
        data = np.expand_dims(data, axis=1)  # モノラルファイルの場合、チャンネルを追加

    # サンプルデータを-1～1の範囲に正規化
    data = data.astype(np.float32) / np.iinfo(np.int16).max
    print(f"Normalized data range: {np.min(data)} to {np.max(data)}")

    # VUメーターフィルタ（約300msのリリースタイム）
    def vu_meter(signal, rate, release_time=0.3):
        alpha = np.exp(-1.0 / (release_time * rate))
        vu_level = np.zeros_like(signal)
        for i in range(1, len(signal)):
            vu_level[i] = max(alpha * vu_level[i-1], np.abs(signal[i]))
        return vu_level

    # VUメーターのレベルを計算
    def calculate_vu_levels(normalized_data):
        vu_levels = np.zeros_like(normalized_data)
        for ch in range(normalized_data.shape[1]):
            vu_levels[:, ch] = vu_meter(normalized_data[:, ch], rate)
        return vu_levels

    # 二分探索でゲインを調整
    def adjust_gain(normalized_data, target_level=TARGET_LEVEL, tol=1e-3):
        low, high = 0.1, 10.0  # 初期のゲイン範囲
        while high - low > tol:
            mid = (low + high) / 2.0
            vu_levels = calculate_vu_levels(normalized_data * mid)
            max_vu = np.max(vu_levels)
            if max_vu < target_level:
                low = mid
            else:
                high = mid
        return (low + high) / 2.0

    gain_adjustment = adjust_gain(data)
    print(f"Gain adjustment factor: {gain_adjustment}")

    # 統一したゲインを適用
    adjusted_data = data * gain_adjustment
    print(f"Adjusted data range: {np.min(adjusted_data)} to {np.max(adjusted_data)}")

    # 新しいファイル名を生成
    output_path = os.path.splitext(file_path)[0] + '_VUoutput.wav'

    # 調整後のオーディオデータを保存
    adjusted_data_int = np.int16(adjusted_data * np.iinfo(np.int16).max)
    wav.write(output_path, rate, adjusted_data_int)
    print(f"Adjusted audio saved as '{output_path}'")

    # プロットで結果を表示
    plt.figure(figsize=(10, 4))
    plt.plot(data[:, 0], label='Original Left')
    if data.shape[1] > 1:
        plt.plot(data[:, 1], label='Original Right')
    plt.plot(adjusted_data[:, 0], label='Adjusted Left')
    if adjusted_data.shape[1] > 1:
        plt.plot(adjusted_data[:, 1], label='Adjusted Right')
    plt.legend()
    plt.title('Audio Waveform')
    plt.show()

    plt.figure(figsize=(10, 4))
    vu_levels = calculate_vu_levels(adjusted_data)
    plt.plot(vu_levels[:, 0], label='VU Levels Left')
    if vu_levels.shape[1] > 1:
        plt.plot(vu_levels[:, 1], label='VU Levels Right')
    plt.axhline(y=REF_LEVEL, color='r', linestyle='--', label='0 VU (-18 dBFS)')
    plt.legend()
    plt.title('VU Meter Levels')
    plt.show()